<a href="https://colab.research.google.com/github/terinkov/DataAnalysis/blob/main/T_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Почему классический t-тест даёт некорректные результаты при неравных дисперсиях групп?**
## Случай одинаковой дисперсии

Пусть имеются две независимые выборки объёмами $n_1$, $n_2$, состоящие из нормально распределённых случайных величин $X_1$ и $X_2$.  
Необходимо проверить по выборочным данным нулевую гипотезу равенства математических ожиданий этих случайных величин:

$$
H_0: M_1 = M_2
$$

В случае, если дисперсии выборок предполагаются одинаковыми, дисперсия разности выборочных средних оценивается как:

$$
V(\Delta) = \sigma^2 \left( \frac{1}{n_1} + \frac{1}{n_2} \right).
$$

Тогда $t$-статистика вычисляется по формуле:

$$
t = \frac{\bar{X}_1 - \bar{X}_2}{s_X \sqrt{\frac{1}{n_1} + \frac{1}{n_2}}},
$$

где $s_X$ — оценка общего стандартного отклонения, вычисляемая как:

$$
s_X = \sqrt{\frac{(n_1 - 1)s_1^2 + (n_2 - 1)s_2^2}{n_1 + n_2 - 2}}.
$$

Эта статистика имеет распределение $t$ с $(n_1 + n_2 - 2)$ степенями свободы.

## Почему возникает ошибка при неравных дисперсиях?

Ошибка возникает из-за неправильной оценки общей дисперсии. При неравенстве дисперсий $S_1^2 \neq S_2^2$ объединенная дисперсия $S_p^2$ больше не является корректной оценкой истинной дисперсии выборочного среднего. Это приводит к следующим проблемам:

* Смещение оценки ошибки  
В знаменателе $t$-статистики используется $S_x$, но если дисперсии разные, то вклад каждой выборки в общую ошибку разный. Однако формула $S_x^2$ предполагает, что обе выборки вносят равный вклад, что не верно. Здесь мы просто не можем вынести из под корня $S_x$ (так как при обоих слагаемых в $t$-статистике в знаменателе разные $S_x(_1)$, $S_x(_2)$)

* Неверное количество степеней свободы  
Формула степеней свободы  

$$
df = n_1 + n_2 - 2
$$  

предполагает, что каждая группа вносит одинаковую неопределенность в оценку дисперсии. Но если дисперсии разные, влияние каждой выборки разное, и эта формула больше не отражает реальное число степеней свободы. Требуется более точная оценка этого параметра, которая производится в $t$-тесте Уэлча.


Из этого всего, распределение $t$-статистики получается с другими параметрами, а не истиными, и p-value мы должны смотреть не по этому распределению, так как здесь параметры не верны. t-тест Уэлча компенсирует указанные ошибки, рассмотрим его ниже





# **2. Как робастные стандартные ошибки (по Уайту) исправляют проблему неравных дисперсий?**

Классический $t$-тест Стьюдента предполагает равенство дисперсий в двух выборках ($S_1^2 = S_2^2$). Если это предположение нарушается, возникает две основные проблемы:  
1. **Смещение оценки стандартной ошибки**, так как объединённая дисперсия $S_p^2$ некорректно усредняет разную изменчивость в выборках.  
2. **Неправильное число степеней свободы**, что ведёт к неверным $p$-значениям и доверительным интервалам.  

### Решение с помощью робастных стандартных ошибок (по Уайту)  
Робастные стандартные ошибки Уайта устраняют проблему за счёт независимой оценки дисперсий в каждой выборке. Вместо объединённой дисперсии используется корректированная оценка стандартной ошибки:

$$
SE_{robust} = \sqrt{\frac{S_1^2}{n_1} + \frac{S_2^2}{n_2}}.
$$

Здесь $S_1^2$ и $S_2^2$ — дисперсии в каждой группе без предположения их равенства.

Далее, $t$-статистика рассчитывается как:

$$
t = \frac{\bar{X}_1 - \bar{X}_2}{SE_{robust}}.
$$

А количество степеней свободы корректируется по формуле Саттерауэйта:

$$
df = \frac{\left( \frac{S_1^2}{n_1} + \frac{S_2^2}{n_2} \right)^2}{\frac{\left(\frac{S_1^2}{n_1}\right)^2}{n_1 - 1} + \frac{\left(\frac{S_2^2}{n_2}\right)^2}{n_2 - 1}}.
$$

### Итог  
Использование робастных стандартных ошибок Уайта позволяет:  
- Корректно учитывать разную изменчивость данных в выборках.  
- Уменьшить смещение оценки стандартной ошибки.  
- Улучшить точность $p$-значений и доверительных интервалов.  
- Сделать тест более надёжным при наличии гетероскедастичности.




# **3. Сравнить p-значения классического теста t-теста и теста Уэлча на данных с разными дисперсиями (код на Python)**

In [ ]:
import numpy as np
import scipy.stats as stats
np.random.seed(42)
# Генерируем данные с разными дисперсиями
group1 = np.random.normal(loc=10, scale=2, size=30)  # Среднее 10, дисперсия 4
group2 = np.random.normal(loc=10, scale=20, size=30)  # Среднее 10, дисперсия 36

# Классический t-тест (предполагает равные дисперсии)
t_stat, p_value_ttest = stats.ttest_ind(group1, group2, equal_var=True)

# Тест Уэлча (не предполагает равенство дисперсий)
t_stat_welch, p_value_welch = stats.ttest_ind(group1, group2, equal_var=False)

print(f"Классический t-тест: p-value = {p_value_ttest:.5f}")
print(f"Тест Уэлча: p-value = {p_value_welch:.5f}")


Классический t-тест: p-value = 0.55133
Тест Уэлча: p-value = 0.55356


# Вывод

Классический $t$-тест дал $p$-value = 0.55133, а тест Уэлча — $p$-value = 0.55356.  
Так как дисперсии выборок различны ($S_1^2 \neq S_2^2$), классический $t$-тест даёт худшие по сравнению с тестом Уэлча результаты. Мы генерировали выборки при верности гипотезы равенства мат ожидания. Чем больше p-value, тем мы охотнее принимаем гипотезу о равенстве мат ожиданий. У t-теста Уэлча p-value больше, чем у классического t-теста, а значит он дает лучший (по построению эксперимента) результат - более вероятное равенство мат ожиданий двух выборок.

При более выраженной разнице в дисперсиях тест Уэлча обеспечит более точные результаты.  

Вывод: если есть сомнения в равенстве дисперсий, лучше использовать **тест Уэлча**.
